In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV,cross_val_predict

from sklearn.metrics import mean_squared_error,r2_score,confusion_matrix,f1_score,precision_recall_curve,roc_curve,roc_auc_score
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_openml
import cv2

# Preparation des données

## Telechargement des données

In [2]:
dataDict=fetch_openml('mnist_784', version=1)

print("Affichage des cles du dictionnaire : ")
for cle in dataDict.keys():
    print(cle)

Affichage des cles du dictionnaire : 
data
target
frame
feature_names
target_names
DESCR
details
categories
url


## Informations sur les données

In [3]:
print("Il y a {} features pour {} échantillons".format(dataDict["data"].shape[1],dataDict["data"].shape[0]))
X=dataDict['data']
print("Il y a {} targets".format(np.unique(dataDict['target']).size))
y=dataDict['target']
y=y.astype('int')
print("Les differentes valeurs sont : {}".format(np.unique(y)))
print("Description détaillée de la base de donnée :")
print(dataDict['DESCR'])

Il y a 784 features pour 70000 échantillons
Il y a 10 targets
Les differentes valeurs sont : [0 1 2 3 4 5 6 7 8 9]
Description détaillée de la base de donnée :
**Author**: Yann LeCun, Corinna Cortes, Christopher J.C. Burges  
**Source**: [MNIST Website](http://yann.lecun.com/exdb/mnist/) - Date unknown  
**Please cite**:  

The MNIST database of handwritten digits with 784 features, raw data available at: http://yann.lecun.com/exdb/mnist/. It can be split in a training set of the first 60,000 examples, and a test set of 10,000 examples  

It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. The original black and white (bilevel) images from NIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The 

Nos valeurs sont donc des Strings

## Repartition des données :

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=1/7,shuffle=False)
print("Il y a bien {} valeurs dans notre base d'apprentissage".format(x_train.shape[0]))


Il y a bien 60000 valeurs dans notre base d'apprentissage


Utilisation du SGDClassifier



In [5]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np



targets_names = [0,1,2,3,4,5,6,7,8,9]


images_train = x_train.astype('float')/255
images_test = x_test.astype('float')/255


In [6]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)))

model.add(tf.keras.layers.Conv2D(64, 
                 kernel_size=(3, 3), 
                 activation='relu'))

model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Conv2D(256, 
                 kernel_size=(3, 3), 
                 activation='relu'))

model.add(tf.keras.layers.Conv2D(256, 
                 kernel_size=(3, 3), 
                 padding='same',
                 activation='relu'))

model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(2048, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(512, activation='relu'))

model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 256)       147712    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 256)       590080    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 256)         0

In [8]:
images_train = images_train.reshape(-1,28,28,1)
images_test = images_test.reshape(-1,28,28,1)

In [9]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [10]:
history = model.fit(images_train,
                    y_train,
                    epochs=10,
                    validation_split=0.2)

Epoch 1/10
1500/1500 [==============================] - 22s 10ms/step - loss: 0.3079 - accuracy: 0.8986 - val_loss: 0.0807 - val_accuracy: 0.9767
Epoch 2/10
1500/1500 [==============================] - 14s 9ms/step - loss: 0.0636 - accuracy: 0.9802 - val_loss: 0.0426 - val_accuracy: 0.9872
Epoch 3/10
1500/1500 [==============================] - 14s 9ms/step - loss: 0.0495 - accuracy: 0.9861 - val_loss: 0.0390 - val_accuracy: 0.9898
Epoch 4/10
1500/1500 [==============================] - 14s 9ms/step - loss: 0.0424 - accuracy: 0.9874 - val_loss: 0.0408 - val_accuracy: 0.9889
Epoch 5/10
1500/1500 [==============================] - 14s 9ms/step - loss: 0.0352 - accuracy: 0.9899 - val_loss: 0.0329 - val_accuracy: 0.9904
Epoch 6/10
1500/1500 [==============================] - 14s 9ms/step - loss: 0.0308 - accuracy: 0.9905 - val_loss: 0.0244 - val_accuracy: 0.9936
Epoch 7/10
1500/1500 [==============================] - 14s 9ms/step - loss: 0.0257 - accuracy: 0.9926 - val_loss: 0.0325 - val_a